# Multi-objective Robust Optimization (MORO)


This exercise demostrates the application of MORO on the lake model. In contrast to the exercises in previous weeks, we will be using a slightly more sophisticated version of the problem. For details see the MORDM assignment for this week.

## Setup MORO

Many objective robust optimization aims at finding decisions that are robust with respect to the various deeply uncertain factors. For this, MORO evalues each candidate decision over a set of scenarios. For each outcome of interest, the robusntess over this set is calculated. A MOEA is used to maximize the robustness. 

For this assignment, we will be using a domain criterion as our robustness metric. The table below lists the rules that you should use for each outcome of interest.

|Outcome of interest| threhsold  |
|-------------------|------------|
| Maximum pollution | $\leq$ 0.75|
| Inertia           | $\geq$ 0.6 |
| Reliability       | $\geq$ 0.99|   
| Utility           | $\geq$ 0.75|

**1) Implement a function for each outcome that takes a numpy array with results for the outcome of interest, and returns the robustness score**

In [87]:
import functools
import numpy as np

def robustness(direction, threshold, data):
    if data.size == 1:
        if direction == SMALLER:
            return float(data <= threshold)
        else:
            return float(data >= threshold)
    else:
        if direction == SMALLER:
            return np.sum(data<=threshold)/data.shape[0]
        else:
            return np.sum(data>=threshold)/data.shape[0]

def maxp(data):
    return np.sum(data<=0.75)/data.shape[0]
    
SMALLER = 'SMALLER'
LARGER = 'LARGER'

maxp = functools.partial(robustness, SMALLER, 0.75)
inertia = functools.partial(robustness, LARGER, 0.6)
reliability = functools.partial(robustness, LARGER, 0.99)
utility = functools.partial(robustness, LARGER, 0.75)




**2) Generate 4 random release policies, and evaluate them over 500 scenarios. Sample the scenarios using Monte Carlo sampling. Next evaulate your robustness function for 1, 2, 3, ... 500 scenarios for each outcome and visualize this. What can you tell about the convergernce of the robusntess metric as a function of the number of scenarios?**

Set up model

In [2]:
import lakemodel_function as lmf
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, ema_logging, Constant, Constraint)
from ema_workbench.em_framework.evaluators import MC
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)

lake_model = Model('Lakemodel', function=lmf.lake_problem)

In [3]:
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,1))]

lake_model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

lake_model.time_horizon = 100 

lake_model.levers = [RealParameter(f"l{i}", 0, 0.1) for i in 
                     range(lake_model.time_horizon)] # we use time_horizon here



Generate 4 random release policies and evaluate them over 500 scenarios

In [40]:
import pandas as pd
n_policies = 4
n_scenarios = 3

all_results = pd.DataFrame()
with MultiprocessingEvaluator(lake_model) as evaluator:
    
    for n_scenarios in [1,2]:
        experiments, results = evaluator.perform_experiments(policies=n_policies, scenarios=n_scenarios, uncertainty_sampling=MC)
        experiments["max_p"] = results["max_P"]
        experiments["utility"] = results["utility"]
        experiments["inertia"] = results["inertia"]
        experiments["reliability"] = results["reliability"]
        experiments["runs"] = n_scenarios
        all_results = pd.concat([all_results, experiments], axis=0)

In [41]:
all_results

,runs,b,delta,mean,q,stdev,l0,l1,l10,l11,...,l97,l98,l99,scenario,policy,model,max_p,utility,inertia,reliability
0,1,0.218902,0.964292,0.014845,3.431888,0.002916,0.026032,0.053935,0.002653,0.028787,...,0.068243,0.067172,0.051622,16,36,Lakemodel,4.933820,0.561993,0.646465,0.2146
1,1,0.218902,0.964292,0.014845,3.431888,0.002916,0.067408,0.021511,0.075803,0.023165,...,0.005714,0.013815,0.078031,16,37,Lakemodel,4.927780,0.515305,0.646465,0.2372
2,1,0.218902,0.964292,0.014845,3.431888,0.002916,0.094055,0.076527,0.073394,0.063593,...,0.047194,0.040016,0.018765,16,38,Lakemodel,4.961476,0.622560,0.707071,0.1715
3,1,0.218902,0.964292,0.014845,3.431888,0.002916,0.001655,0.037639,0.044761,0.090732,...,0.091404,0.090221,0.028793,16,39,Lakemodel,4.902283,0.489017,0.595960,0.3402
0,2,0.436235,0.932618,0.020389,2.313795,0.002408,0.075523,0.087156,0.005869,0.027055,...,0.010223,0.056653,0.022555,17,40,Lakemodel,0.343040,0.315825,0.636364,1.0000
1,2,0.135476,0.976876,0.010157,3.430210,0.004504,0.075523,0.087156,0.005869,0.027055,...,0.010223,0.056653,0.022555,18,40,Lakemodel,7.895223,0.727601,0.636364,0.0801
2,2,0.436235,0.932618,0.020389,2.313795,0.002408,0.028994,0.043730,0.064643,0.009475,...,0.083460,0.029821,0.052674,17,41,Lakemodel,2.207955,0.307257,0.707071,0.5353
3,2,0.135476,0.976876,0.010157,3.430210,0.004504,0.028994,0.043730,0.064643,0.009475,...,0.083460,0.029821,0.052674,18,41,Lakemodel,7.986678,0.842318,0.707071,0.1084
4,2,0.436235,0.932618,0.020389,2.313795,0.002408,0.002730,0.000835,0.078022,0.092892,...,0.041843,0.085482,0.034334,17,42,Lakemodel,1.401228,0.266691,0.606061,0.8575
5,2,0.135476,0.976876,0.010157,3.430210,0.004504,0.002730,0.000835,0.078022,0.092892,...,0.041843,0.085482,0.034334,18,42,Lakemodel,7.894881,0.758615,0.606061,0.1135


In [46]:
grouped = all_results.groupby(by=["runs"])

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [60]:
for group in grouped:

    mres =maxp(group[1]["max_p"])
    
    print(group)

0.0
(1,   runs         b     delta      mean         q     stdev        l0        l1  \
0    1  0.218902  0.964292  0.014845  3.431888  0.002916  0.026032  0.053935   
1    1  0.218902  0.964292  0.014845  3.431888  0.002916  0.067408  0.021511   
2    1  0.218902  0.964292  0.014845  3.431888  0.002916  0.094055  0.076527   
3    1  0.218902  0.964292  0.014845  3.431888  0.002916  0.001655  0.037639   

        l10       l11  ...       l97       l98       l99  scenario  policy  \
0  0.002653  0.028787  ...  0.068243  0.067172  0.051622        16      36   
1  0.075803  0.023165  ...  0.005714  0.013815  0.078031        16      37   
2  0.073394  0.063593  ...  0.047194  0.040016  0.018765        16      38   
3  0.044761  0.090732  ...  0.091404  0.090221  0.028793        16      39   

       model     max_p   utility   inertia  reliability  
0  Lakemodel  4.933820  0.561993  0.646465       0.2146  
1  Lakemodel  4.927780  0.515305  0.646465       0.2372  
2  Lakemodel  4.961476  0.

## Searching for candidate solutions
Set up the robust optimization problem using the robustness functions you have specified. Assume that you will need 50 scenarios for estimating the robustness. Use $\epsilon$-progress and hypervolume to track convergence. Solve the optimization problem. As $\epsilon$ values, you can assume 0.05 for each of the four robustness metrics.

*note: this optimization problem is computationally very expensive. Develop and test your code using a sequential evaluator, a low number of function evaluations (e.g., 200), and a low number of scenarios (e.g., 5). Once everything seems to be working replace the sequential evaluator with an multiprocessing or ipyparallel evaluator, and increase the number of nfe and scenarios*.


**Plot your $\epsilon$-progress to evaluate convergergence, and visualize the trade-offs using parallel coordinate plots**

In [4]:
from ema_workbench import SequentialEvaluator

In [86]:
nfe = 10
epsilons = [0.05] * 4
n_scenarios = 5

MAXIMIZE = ScalarOutcome.MAXIMIZE
MINIMIZE = ScalarOutcome.MINIMIZE

convergence_metrics = [HyperVolume.from_outcomes(lake_model.outcomes),
                       EpsilonProgress()]

constraints = [Constraint("max pollution", outcome_names="max_P", function=maxp),
              Constraint("inertia", outcome_names="inertia", function=inertia),
              Constraint("reliability", outcome_names="reliability", function=reliability),
              Constraint("utility", outcome_names="utility", function=utility)]


robustnes_functions = [ScalarOutcome('max_P', kind=MINIMIZE,
                             variable_name='max_P', function=maxp),
                       ScalarOutcome('reliability', kind=MAXIMIZE,
                             variable_name='reliability', function=reliability),
                       ScalarOutcome('inertia', kind=MAXIMIZE,
                             variable_name='inertia', function=inertia),
                       ScalarOutcome('utility', kind=MAXIMIZE,
                             variable_name='utility', function=utility)]


with SequentialEvaluator(lake_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robustnes_functions, n_scenarios, nfe=nfe, epsilons=epsilons, convergence=convergence_metrics, constraints=constraints)

KeyboardInterrupt: 

In [81]:
results

,l0,l1,l2,l3,l4,l5,l6,l7,l8,l9,...,l94,l95,l96,l97,l98,l99,max_p,reliability,inertia,utility
0,0.018490,0.031633,0.062429,0.013053,0.084848,0.037411,0.069415,0.097772,0.040826,0.082859,...,0.058370,0.038525,0.013059,0.011189,0.095639,0.064032,0.4,0.4,1.0,0.2
1,0.026318,0.074756,0.042366,0.065882,0.015139,0.022555,0.019469,0.051755,0.034421,0.040640,...,0.003147,0.007824,0.055264,0.059599,0.020913,0.089155,0.2,0.2,1.0,0.2
2,0.009929,0.036563,0.097142,0.017356,0.064713,0.089878,0.077072,0.071795,0.065950,0.081097,...,0.068088,0.016587,0.042003,0.053798,0.099213,0.045824,0.0,0.0,1.0,0.4


**What does this plot tell us about the tradeoffs and conflicting objectives?**

## Re-evaluate candidate solutions under uncertainty

We have used only 50 scenarios for the optimization. Take the results and re-evaluate them over a larger set (assume 1000 scenarios). How different are your results? What does this imply for the assumption of 50 scenarios during robust optimization.

*hint: use the to_dict method on a dataframe, next generate Policy objects in a list expression by iterating over the dicts returned by the to_dict method*

## Comparison
If you have time, import your solutions found for MORDM and re-evaluate them over the same set of scnearios as used for re-evaluating the MORO results. Compare the robustness of MORDM and MORO, what do you observe?